In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    DRIVE_MOUNTED = True
    print("✅ Google Drive mounted!")
except:
    DRIVE_MOUNTED = False
    print("ℹ️ Google Drive not available")

Mounted at /content/drive
✅ Google Drive mounted!


In [3]:
! pip install openai-clip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=aad99ff2f5de16f2bee95584398169380d98aeb526af7c1b348ff5dacb50cfdd
  Stored in directory: /root/.cache/pip/wheels/0d/17/90/042948fd2e2a87f1dcf6db6d438cad015c49db0c53d1d9c7dc
Successfully built openai-clip


In [ ]:
! pip install pytesseract

In [ ]:
# ====================================================================
# GREENTEXT CLIP PREPROCESSING PIPELINE - FIXED VERSION
# Purpose: Preprocess scraped greentext data for CLIP model training
# ====================================================================

import pandas as pd
import numpy as np
import torch
import clip
from PIL import Image, ImageEnhance, ImageFilter
import cv2
import pytesseract
import re
import json
import os
from pathlib import Path
from typing import List, Dict, Tuple, Optional
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import warnings
warnings.filterwarnings('ignore')

# Install required packages
import subprocess
import sys

def install_packages():
    packages = [
        'opencv-python', 'pytesseract', 'transformers',
        'torch', 'torchvision', 'ftfy', 'regex', 'tqdm',
        'scikit-learn', 'matplotlib', 'seaborn'
    ]
    for package in packages:
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package, '-q'])
        except:
            print(f"⚠️ Could not install {package}")

    # Install CLIP
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'git+https://github.com/openai/CLIP.git', '-q'])
        print("✅ CLIP installed successfully!")
    except:
        print("❌ Failed to install CLIP")

print("📦 Installing required packages...")
install_packages()

class GreentextCLIPPreprocessor:
    """Complete preprocessing pipeline for greentext images and CLIP training"""

    def __init__(self, dataset_path: str, images_path: str):
        """
        Initialize the preprocessor

        Args:
            dataset_path: Path to the CSV file with scraped data
            images_path: Path to the directory containing downloaded images
        """
        self.dataset_path = Path(dataset_path)
        self.images_path = Path(images_path)
        self.processed_path = self.dataset_path.parent / "processed"
        self.processed_path.mkdir(exist_ok=True)

        # Load dataset
        self.df = pd.read_csv(dataset_path)
        print(f"📊 Loaded dataset with {len(self.df)} records")

        # Initialize CLIP model for preprocessing
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"🖥️ Using device: {self.device}")

        try:
            self.clip_model, self.clip_preprocess = clip.load("ViT-B/32", device=self.device)
            print("✅ CLIP model loaded successfully")
        except Exception as e:
            print(f"❌ Failed to load CLIP: {e}")
            self.clip_model = None

    def analyze_dataset_quality(self):
        """Analyze the quality and characteristics of the scraped dataset"""
        print("\n📊 DATASET QUALITY ANALYSIS")
        print("=" * 50)

        # Basic statistics
        successful_downloads = self.df[self.df['download_success'] == True]
        print(f"✅ Successfully downloaded images: {len(successful_downloads)}")
        print(f"❌ Failed downloads: {len(self.df) - len(successful_downloads)}")

        if len(successful_downloads) == 0:
            print("⚠️ No successful downloads found!")
            return None

        # Score distribution
        print(f"\n📈 Score Statistics:")
        print(f"   Average score: {successful_downloads['score'].mean():.1f}")
        print(f"   Median score: {successful_downloads['score'].median():.1f}")
        print(f"   Score range: {successful_downloads['score'].min()} - {successful_downloads['score'].max()}")

        # Image dimensions analysis
        if 'dimensions' in successful_downloads.columns:
            dims = successful_downloads['dimensions'].apply(eval)  # Convert string to tuple
            widths = dims.apply(lambda x: x[0])
            heights = dims.apply(lambda x: x[1])

            print(f"\n🖼️ Image Dimensions:")
            print(f"   Average width: {widths.mean():.0f}px")
            print(f"   Average height: {heights.mean():.0f}px")
            print(f"   Width range: {widths.min()} - {widths.max()}")
            print(f"   Height range: {heights.min()} - {heights.max()}")

        # Title analysis
        title_lengths = successful_downloads['title'].str.len()
        print(f"\n📝 Title Analysis:")
        print(f"   Average title length: {title_lengths.mean():.1f} characters")
        print(f"   Title length range: {title_lengths.min()} - {title_lengths.max()}")

        return successful_downloads

    def extract_text_from_image(self, image_path: str) -> Dict[str, str]:
        """
        Extract text from greentext image using OCR

        Args:
            image_path: Path to the image file

        Returns:
            Dictionary containing extracted text and metadata
        """
        try:
            # Load image
            image = cv2.imread(str(image_path))
            if image is None:
                return {"text": "", "confidence": 0, "error": "Could not load image"}

            # Preprocess image for better OCR
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Enhance contrast
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
            enhanced = clahe.apply(gray)

            # Apply gaussian blur to reduce noise
            blurred = cv2.GaussianBlur(enhanced, (1, 1), 0)

            # Threshold to get binary image
            _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # OCR configuration for greentext
            custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz.,!?;:()[]{}"\'-/\n >'

            # Extract text
            text = pytesseract.image_to_string(thresh, config=custom_config)

            # Get confidence scores
            try:
                data = pytesseract.image_to_data(thresh, output_type=pytesseract.Output.DICT)
                confidences = [int(conf) for conf in data['conf'] if int(conf) > 0]
                avg_confidence = np.mean(confidences) if confidences else 0
            except:
                avg_confidence = 0

            # Clean extracted text
            cleaned_text = self.clean_extracted_text(text)

            return {
                "text": cleaned_text,
                "raw_text": text,
                "confidence": avg_confidence,
                "text_length": len(cleaned_text),
                "error": None
            }

        except Exception as e:
            return {"text": "", "confidence": 0, "error": str(e)}

    def clean_extracted_text(self, text: str) -> str:
        """Clean and normalize extracted text from greentext images"""
        if not text:
            return ""

        # Remove extra whitespace and newlines
        text = re.sub(r'\s+', ' ', text)

        # Remove common OCR artifacts
        text = re.sub(r'[^\w\s.,!?;:()\[\]{}"\'>/\-]', '', text)

        # Fix common greentext patterns
        text = re.sub(r'>\s*([a-zA-Z])', r'> \1', text)  # Fix greentext arrows
        text = re.sub(r'be\s+me', 'be me', text, flags=re.IGNORECASE)  # Fix common phrase

        # Remove lines that are too short (likely OCR errors)
        lines = text.split('\n')
        meaningful_lines = [line.strip() for line in lines if len(line.strip()) > 3]

        return '\n'.join(meaningful_lines).strip()

    def create_text_embeddings(self, texts: List[str], batch_size: int = 32) -> np.ndarray:
        """Create CLIP text embeddings for the extracted text"""
        if not self.clip_model:
            print("❌ CLIP model not available")
            return np.array([])

        all_embeddings = []

        print("🔄 Creating text embeddings...")
        for i in tqdm(range(0, len(texts), batch_size)):
            batch_texts = texts[i:i+batch_size]

            # Tokenize and encode
            try:
                text_tokens = clip.tokenize(batch_texts, truncate=True).to(self.device)

                with torch.no_grad():
                    text_embeddings = self.clip_model.encode_text(text_tokens)
                    text_embeddings = text_embeddings / text_embeddings.norm(dim=-1, keepdim=True)

                all_embeddings.append(text_embeddings.cpu().numpy())

            except Exception as e:
                print(f"❌ Error processing batch {i//batch_size}: {e}")
                # Add zero embeddings for failed batch
                zero_embeddings = np.zeros((len(batch_texts), 512))
                all_embeddings.append(zero_embeddings)

        return np.vstack(all_embeddings)

    def create_image_embeddings(self, image_paths: List[str], batch_size: int = 16) -> np.ndarray:
        """Create CLIP image embeddings for the greentext images"""
        if not self.clip_model:
            print("❌ CLIP model not available")
            return np.array([])

        all_embeddings = []

        print("🔄 Creating image embeddings...")
        for i in tqdm(range(0, len(image_paths), batch_size)):
            batch_paths = image_paths[i:i+batch_size]
            batch_images = []

            # Load and preprocess images
            for path in batch_paths:
                try:
                    image = Image.open(path).convert('RGB')
                    processed_image = self.clip_preprocess(image)
                    batch_images.append(processed_image)
                except Exception as e:
                    print(f"❌ Error loading image {path}: {e}")
                    # Add a blank image
                    blank_image = Image.new('RGB', (224, 224), color='white')
                    processed_image = self.clip_preprocess(blank_image)
                    batch_images.append(processed_image)

            # Create embeddings
            try:
                image_batch = torch.stack(batch_images).to(self.device)

                with torch.no_grad():
                    image_embeddings = self.clip_model.encode_image(image_batch)
                    image_embeddings = image_embeddings / image_embeddings.norm(dim=-1, keepdim=True)

                all_embeddings.append(image_embeddings.cpu().numpy())

            except Exception as e:
                print(f"❌ Error processing image batch {i//batch_size}: {e}")
                # Add zero embeddings for failed batch
                zero_embeddings = np.zeros((len(batch_paths), 512))
                all_embeddings.append(zero_embeddings)

        return np.vstack(all_embeddings)

    def enhance_titles_with_context(self, df: pd.DataFrame) -> pd.DataFrame:
        """Enhance titles with additional context for better search"""
        df = df.copy()

        # Create enhanced descriptions
        enhanced_descriptions = []

        for _, row in df.iterrows():
            description_parts = []

            # Add title
            if pd.notna(row['title']):
                description_parts.append(row['title'])

            # Add score context
            if row['score'] > 1000:
                description_parts.append("popular greentext")
            elif row['score'] > 500:
                description_parts.append("well-liked greentext")

            # Add engagement context
            if row['engagement_ratio'] > 0.1:
                description_parts.append("highly discussed")

            # Add extracted text if available
            if 'extracted_text' in row and pd.notna(row['extracted_text']) and row['extracted_text']:
                # Add first few words of extracted text
                text_preview = row['extracted_text'][:200] + "..." if len(row['extracted_text']) > 200 else row['extracted_text']
                description_parts.append(text_preview)

            enhanced_descriptions.append(" | ".join(description_parts))

        df['enhanced_description'] = enhanced_descriptions
        return df

    def create_search_index(self, df: pd.DataFrame, text_embeddings: np.ndarray,
                          image_embeddings: np.ndarray) -> Dict:
        """Create a search index for fast similarity search"""

        # Combine text and image embeddings (weighted average)
        text_weight = 0.3
        image_weight = 0.7

        combined_embeddings = (text_weight * text_embeddings +
                             image_weight * image_embeddings)

        # Normalize combined embeddings
        combined_embeddings = combined_embeddings / np.linalg.norm(combined_embeddings, axis=1, keepdims=True)

        search_index = {
            'embeddings': combined_embeddings,
            'text_embeddings': text_embeddings,
            'image_embeddings': image_embeddings,
            'metadata': df[['post_id', 'title', 'score', 'filename', 'enhanced_description']].to_dict('records'),
            'text_weight': text_weight,
            'image_weight': image_weight
        }

        return search_index

    def run_complete_preprocessing(self):
        """Run the complete preprocessing pipeline"""
        print("🚀 STARTING GREENTEXT CLIP PREPROCESSING PIPELINE")
        print("=" * 60)

        # Step 1: Analyze dataset quality
        valid_df = self.analyze_dataset_quality()
        if valid_df is None or len(valid_df) == 0:
            print("❌ No valid data to process")
            return None

        # Make a copy to avoid modifying original
        valid_df = valid_df.copy()

        # Step 2: Extract text from images
        print("\n🔍 EXTRACTING TEXT FROM IMAGES")
        print("-" * 40)

        text_data = []
        image_paths = []

        for _, row in tqdm(valid_df.iterrows(), total=len(valid_df), desc="Processing images"):
            image_path = self.images_path / row['filename']

            if image_path.exists():
                # Extract text
                text_result = self.extract_text_from_image(image_path)
                text_data.append(text_result)
                image_paths.append(str(image_path))
            else:
                print(f"⚠️ Image not found: {image_path}")
                text_data.append({"text": "", "confidence": 0, "error": "File not found"})
                image_paths.append("")

        # FIXED: Add extracted text columns properly
        text_df = pd.DataFrame(text_data)
        for col in text_df.columns:
            valid_df[f'extracted_{col}'] = text_df[col].values

        # Step 3: Enhance descriptions
        print("\n📝 ENHANCING DESCRIPTIONS")
        print("-" * 40)
        valid_df = self.enhance_titles_with_context(valid_df)

        # Step 4: Create embeddings
        print("\n🧠 CREATING EMBEDDINGS")
        print("-" * 40)

        # Prepare texts for embedding
        texts_for_embedding = valid_df['enhanced_description'].fillna('').tolist()
        valid_image_paths = [path for path in image_paths if path and os.path.exists(path)]

        # Create text embeddings
        text_embeddings = self.create_text_embeddings(texts_for_embedding)

        # Create image embeddings
        image_embeddings = self.create_image_embeddings(valid_image_paths)

        # Step 5: Create search index
        print("\n🔍 CREATING SEARCH INDEX")
        print("-" * 40)

        search_index = self.create_search_index(valid_df, text_embeddings, image_embeddings)

        # Step 6: Save processed data
        print("\n💾 SAVING PROCESSED DATA")
        print("-" * 40)

        # Save processed dataframe
        processed_csv_path = self.processed_path / "processed_greentext_data.csv"
        valid_df.to_csv(processed_csv_path, index=False)
        print(f"✅ Saved processed data: {processed_csv_path}")

        # Save embeddings
        embeddings_path = self.processed_path / "embeddings.npz"
        np.savez_compressed(embeddings_path,
                          text_embeddings=text_embeddings,
                          image_embeddings=image_embeddings,
                          combined_embeddings=search_index['embeddings'])
        print(f"✅ Saved embeddings: {embeddings_path}")

        # Save search index
        index_path = self.processed_path / "search_index.pkl"
        with open(index_path, 'wb') as f:
            pickle.dump(search_index, f)
        print(f"✅ Saved search index: {index_path}")

        # Create train/test split for ML training
        if len(valid_df) >= 10:
            train_df, test_df = train_test_split(valid_df, test_size=0.2, random_state=42)

            train_path = self.processed_path / "train_data.csv"
            test_path = self.processed_path / "test_data.csv"

            train_df.to_csv(train_path, index=False)
            test_df.to_csv(test_path, index=False)

            print(f"✅ Saved train set: {train_path} ({len(train_df)} samples)")
            print(f"✅ Saved test set: {test_path} ({len(test_df)} samples)")

        # Step 7: Generate summary
        print("\n📊 PREPROCESSING SUMMARY")
        print("=" * 50)
        print(f"✅ Total images processed: {len(valid_df)}")
        print(f"🔤 Text extracted from: {len([d for d in text_data if d['text']])}")
        print(f"🧠 Text embeddings created: {text_embeddings.shape}")
        print(f"🖼️ Image embeddings created: {image_embeddings.shape}")
        print(f"🔍 Search index ready with {len(search_index['metadata'])} items")
        print(f"💾 All data saved to: {self.processed_path}")

        return {
            'processed_df': valid_df,
            'text_embeddings': text_embeddings,
            'image_embeddings': image_embeddings,
            'search_index': search_index,
            'processed_path': self.processed_path
        }

# ============== USAGE EXAMPLE ==============
def run_preprocessing_pipeline(dataset_csv_path: str, images_directory: str):
    """
    Run the complete preprocessing pipeline

    Args:
        dataset_csv_path: Path to your scraped dataset CSV
        images_directory: Path to directory containing downloaded images
    """

    print("🔥 GREENTEXT CLIP PREPROCESSING")
    print("=" * 50)

    try:
        # Initialize preprocessor
        preprocessor = GreentextCLIPPreprocessor(dataset_csv_path, images_directory)

        # Run complete pipeline
        results = preprocessor.run_complete_preprocessing()

        if results:
            print("\n🎉 PREPROCESSING COMPLETED SUCCESSFULLY!")
            print("🚀 Your data is now ready for CLIP-based search!")
            print("\n📋 Next steps:")
            print("1. Use the search_index.pkl for building your search engine")
            print("2. Use processed_greentext_data.csv for additional ML tasks")
            print("3. Use train_data.csv and test_data.csv for model training")

            return results
        else:
            print("❌ Preprocessing failed")
            return None

    except Exception as e:
        print(f"❌ Error during preprocessing: {e}")
        import traceback
        traceback.print_exc()
        return None

# ============== CONFIGURATION ==============
# Update these paths to match your scraped data
DATASET_CSV_PATH = "/content/drive/MyDrive/greentext_ml_dataset/metadata/greentext_complete.csv"  # Path to your scraped CSV
IMAGES_DIRECTORY = "/content/drive/MyDrive/greentext_ml_dataset/images"  # Path to your images folder

# ============== RUN PIPELINE ==============
if __name__ == "__main__":
    print("🚀 Starting CLIP preprocessing pipeline...")
    results = run_preprocessing_pipeline(DATASET_CSV_PATH, IMAGES_DIRECTORY)

    if results:
        print(f"\n✨ Preprocessing completed! Check {results['processed_path']} for output files.")
    else:
        print("\n❌ Preprocessing failed. Please check the error messages above.")

📦 Installing required packages...
✅ CLIP installed successfully!
🚀 Starting CLIP preprocessing pipeline...
🔥 GREENTEXT CLIP PREPROCESSING
📊 Loaded dataset with 898 records
🖥️ Using device: cuda
✅ CLIP model loaded successfully
🚀 STARTING GREENTEXT CLIP PREPROCESSING PIPELINE

📊 DATASET QUALITY ANALYSIS
✅ Successfully downloaded images: 898
❌ Failed downloads: 0

📈 Score Statistics:
   Average score: 7089.0
   Median score: 7447.0
   Score range: 52 - 49570

🖼️ Image Dimensions:
   Average width: 861px
   Average height: 935px
   Width range: 161 - 4640
   Height range: 81 - 4586

📝 Title Analysis:
   Average title length: 22.9 characters
   Title length range: 2 - 84

🔍 EXTRACTING TEXT FROM IMAGES
----------------------------------------


Processing images:   0%|          | 0/898 [00:00<?, ?it/s]


📝 ENHANCING DESCRIPTIONS
----------------------------------------

🧠 CREATING EMBEDDINGS
----------------------------------------
🔄 Creating text embeddings...


  0%|          | 0/29 [00:00<?, ?it/s]

🔄 Creating image embeddings...


  0%|          | 0/57 [00:00<?, ?it/s]


🔍 CREATING SEARCH INDEX
----------------------------------------

💾 SAVING PROCESSED DATA
----------------------------------------
✅ Saved processed data: /content/drive/MyDrive/greentext_ml_dataset/metadata/processed/processed_greentext_data.csv
✅ Saved embeddings: /content/drive/MyDrive/greentext_ml_dataset/metadata/processed/embeddings.npz
✅ Saved search index: /content/drive/MyDrive/greentext_ml_dataset/metadata/processed/search_index.pkl
✅ Saved train set: /content/drive/MyDrive/greentext_ml_dataset/metadata/processed/train_data.csv (718 samples)
✅ Saved test set: /content/drive/MyDrive/greentext_ml_dataset/metadata/processed/test_data.csv (180 samples)

📊 PREPROCESSING SUMMARY
✅ Total images processed: 898
🔤 Text extracted from: 0
🧠 Text embeddings created: (898, 512)
🖼️ Image embeddings created: (898, 512)
🔍 Search index ready with 898 items
💾 All data saved to: /content/drive/MyDrive/greentext_ml_dataset/metadata/processed

🎉 PREPROCESSING COMPLETED SUCCESSFULLY!
🚀 Your data i

In [4]:
# ====================================================================
# COLAB SETUP SCRIPT FOR GREENTEXT SEARCH
# Run this cell first to install dependencies and setup the environment
# ====================================================================

import subprocess
import sys
import os

def install_requirements():
    """Install required packages for the search engine"""

    print("📦 INSTALLING REQUIRED PACKAGES")
    print("=" * 50)

    # Core packages
    packages = [
        'faiss-cpu',  # For similarity search
        'ipywidgets',  # For interactive widgets
        'flask',  # For optional API server
        'flask-cors',  # For API CORS
        'matplotlib',  # For displaying results
        'ftfy',  # Text cleaning
        'regex',  # Regular expressions
    ]

    for package in packages:
        try:
            print(f"Installing {package}...")
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package, '-q'])
            print(f"✅ {package} installed successfully")
        except Exception as e:
            print(f"❌ Failed to install {package}: {e}")

    # Install CLIP
    try:
        print("Installing OpenAI CLIP...")
        subprocess.check_call([
            sys.executable, '-m', 'pip', 'install',
            'git+https://github.com/openai/CLIP.git', '-q'
        ])
        print("✅ CLIP installed successfully")
    except Exception as e:
        print(f"❌ Failed to install CLIP: {e}")

    print("\n🎉 All packages installed!")

def setup_colab_environment():
    """Setup Colab environment for the search engine"""

    print("\n🔧 SETTING UP COLAB ENVIRONMENT")
    print("=" * 50)

    # Enable widgets
    try:
        from google.colab import output
        output.enable_custom_widget_manager()
        print("✅ Widgets enabled")
    except:
        print("⚠️ Could not enable widgets (not in Colab environment)")

    # Mount Google Drive
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        print("✅ Google Drive mounted")
    except:
        print("⚠️ Could not mount Google Drive")

    # Check if processed data exists
    processed_path = "/content/drive/MyDrive/greentext_ml_dataset/metadata/processed"
    images_path = "/content/drive/MyDrive/greentext_ml_dataset/images"

    if os.path.exists(processed_path):
        print(f"✅ Found processed data at: {processed_path}")

        # List files in processed directory
        files = os.listdir(processed_path)
        print(f"📁 Files in processed directory: {files}")

        # Check for required files
        required_files = [
            'processed_greentext_data.csv',
            'embeddings.npz',
            'search_index.pkl'
        ]

        missing_files = [f for f in required_files if f not in files]
        if missing_files:
            print(f"⚠️ Missing required files: {missing_files}")
            print("Please run the preprocessing pipeline first!")
        else:
            print("✅ All required files found!")

    else:
        print(f"❌ Processed data not found at: {processed_path}")
        print("Please run the preprocessing pipeline first!")

    if os.path.exists(images_path):
        image_count = len([f for f in os.listdir(images_path) if f.endswith(('.jpg', '.jpeg', '.png', '.gif'))])
        print(f"✅ Found {image_count} images at: {images_path}")
    else:
        print(f"❌ Images not found at: {images_path}")

def check_gpu():
    """Check GPU availability"""

    print("\n🖥️ CHECKING GPU AVAILABILITY")
    print("=" * 50)

    import torch

    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"✅ GPU Available: {gpu_name}")
        print(f"💾 GPU Memory: {gpu_memory:.1f} GB")

        # Test GPU
        try:
            test_tensor = torch.randn(1000, 1000).cuda()
            result = torch.matmul(test_tensor, test_tensor.T)
            print("✅ GPU test successful")
        except Exception as e:
            print(f"❌ GPU test failed: {e}")

    else:
        print("⚠️ No GPU available - using CPU")
        print("This will be slower but still functional")

def display_next_steps():
    """Display instructions for next steps"""

    print("\n🚀 NEXT STEPS")
    print("=" * 50)
    print("1. If you haven't run the preprocessing pipeline yet:")
    print("   - Upload your scraped greentext data")
    print("   - Run the preprocessing script")
    print()
    print("2. If preprocessing is complete, run the search engine:")
    print("   - Copy and run the search engine code")
    print("   - Use the interactive search interface")
    print()
    print("3. Example search queries to try:")
    print("   - 'wholesome story'")
    print("   - 'anon at work'")
    print("   - 'mom greentext'")
    print("   - 'autism story'")
    print("   - 'feels thread'")
    print()
    print("🎉 Setup complete! You're ready to search greentexts!")

# ============== MAIN SETUP FUNCTION ==============

def run_complete_setup():
    """Run the complete setup process"""

    print("🚀 GREENTEXT SEARCH SETUP")
    print("=" * 60)
    print("This will prepare your Colab environment for greentext search")
    print("=" * 60)

    # Install packages
    install_requirements()

    # Setup environment
    setup_colab_environment()

    # Check GPU
    check_gpu()

    # Show next steps
    display_next_steps()

    print("\n" + "=" * 60)
    print("✨ SETUP COMPLETE! Ready to search greentexts! ✨")
    print("=" * 60)

# Run setup automatically
if __name__ == "__main__":
    run_complete_setup()
else:
    # Auto-run when imported
    run_complete_setup()

🚀 GREENTEXT SEARCH SETUP
This will prepare your Colab environment for greentext search
📦 INSTALLING REQUIRED PACKAGES
Installing faiss-cpu...
✅ faiss-cpu installed successfully
Installing ipywidgets...
✅ ipywidgets installed successfully
Installing flask...
✅ flask installed successfully
Installing flask-cors...
✅ flask-cors installed successfully
Installing matplotlib...
✅ matplotlib installed successfully
Installing ftfy...
✅ ftfy installed successfully
Installing regex...
✅ regex installed successfully
Installing OpenAI CLIP...
✅ CLIP installed successfully

🎉 All packages installed!

🔧 SETTING UP COLAB ENVIRONMENT
✅ Widgets enabled
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted
✅ Found processed data at: /content/drive/MyDrive/greentext_ml_dataset/metadata/processed
📁 Files in processed directory: ['processed_greentext_data.csv', 'embeddings.npz', 'search_index.pkl', 'train_data

In [5]:
# ====================================================================
# GREENTEXT SEARCH ENGINE FOR GOOGLE COLAB
# Complete search interface that runs in Colab notebooks
# ====================================================================

import numpy as np
import pandas as pd
import pickle
import clip
import torch
from typing import List, Dict, Optional
import faiss
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import threading
import time
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import base64
import io
import requests
from threading import Thread
import logging

# Disable matplotlib warnings
import warnings
warnings.filterwarnings('ignore')

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ColabGreentextSearch:
    """Greentext search engine optimized for Google Colab"""

    def __init__(self, processed_data_path: str, images_path: str):
        """Initialize the search engine"""
        self.processed_data_path = Path(processed_data_path)
        self.images_path = Path(images_path)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        print(f"🖥️ Using device: {self.device}")

        # Load CLIP model
        print("📦 Loading CLIP model...")
        self.clip_model, self.clip_preprocess = clip.load("ViT-B/32", device=self.device)
        print("✅ CLIP model loaded successfully")

        # Load processed data
        self.load_processed_data()

        # Build FAISS indices
        self.build_faiss_indices()

        print("🚀 Search engine ready!")

    def load_processed_data(self):
        """Load processed data and embeddings"""
        try:
            # Load metadata
            csv_path = self.processed_data_path / "processed_greentext_data.csv"
            self.df = pd.read_csv(csv_path)
            print(f"📊 Loaded {len(self.df)} records")

            # Load embeddings
            embeddings_path = self.processed_data_path / "embeddings.npz"
            embeddings_data = np.load(embeddings_path)

            self.text_embeddings = embeddings_data['text_embeddings']
            self.image_embeddings = embeddings_data['image_embeddings']
            self.combined_embeddings = embeddings_data['combined_embeddings']

            print(f"🧠 Loaded embeddings - Shape: {self.text_embeddings.shape}")

        except Exception as e:
            print(f"❌ Error loading processed data: {e}")
            raise

    def build_faiss_indices(self):
        """Build FAISS indices for fast similarity search"""
        try:
            d = self.text_embeddings.shape[1]  # dimension

            # Combined embeddings index (primary)
            self.combined_index = faiss.IndexFlatIP(d)
            self.combined_index.add(self.combined_embeddings.astype('float32'))

            # Text embeddings index
            self.text_index = faiss.IndexFlatIP(d)
            self.text_index.add(self.text_embeddings.astype('float32'))

            # Image embeddings index
            self.image_index = faiss.IndexFlatIP(d)
            self.image_index.add(self.image_embeddings.astype('float32'))

            print("🔍 FAISS indices built successfully")

        except Exception as e:
            print(f"❌ Error building FAISS indices: {e}")
            raise

    def encode_text_query(self, query: str) -> np.ndarray:
        """Encode text query using CLIP"""
        try:
            text_tokens = clip.tokenize([query], truncate=True).to(self.device)

            with torch.no_grad():
                text_embedding = self.clip_model.encode_text(text_tokens)
                text_embedding = text_embedding / text_embedding.norm(dim=-1, keepdim=True)

            return text_embedding.cpu().numpy().astype('float32')

        except Exception as e:
            print(f"❌ Error encoding text query: {e}")
            raise

    def search(self, query: str, top_k: int = 12, search_type: str = "combined") -> List[Dict]:
        """Perform semantic search"""
        try:
            # Encode query
            query_embedding = self.encode_text_query(query)

            # Choose appropriate index
            if search_type == "text":
                index = self.text_index
            elif search_type == "image":
                index = self.image_index
            else:  # combined
                index = self.combined_index

            # Search
            similarities, indices = index.search(query_embedding, top_k)

            # Prepare results
            results = []
            for similarity, idx in zip(similarities[0], indices[0]):
                if idx < len(self.df):
                    row = self.df.iloc[idx]

                    result = {
                        "post_id": str(row['post_id']),
                        "title": row['title'],
                        "score": int(row['score']),
                        "similarity_score": float(similarity),
                        "filename": row['filename'],
                        "enhanced_description": row.get('enhanced_description', ''),
                        "image_path": str(self.images_path / row['filename'])
                    }
                    results.append(result)

            return results

        except Exception as e:
            print(f"❌ Error during search: {e}")
            return []

    def get_stats(self) -> Dict:
        """Get dataset statistics"""
        stats = {
            "total_images": len(self.df),
            "average_score": float(self.df['score'].mean()),
            "max_score": int(self.df['score'].max()),
            "min_score": int(self.df['score'].min()),
            "median_score": float(self.df['score'].median()),
            "embedding_dimensions": self.text_embeddings.shape[1]
        }
        return stats

    def get_random_samples(self, n: int = 6) -> List[Dict]:
        """Get random samples from the dataset"""
        random_indices = np.random.choice(len(self.df), size=min(n, len(self.df)), replace=False)

        results = []
        for idx in random_indices:
            row = self.df.iloc[idx]
            result = {
                "post_id": str(row['post_id']),
                "title": row['title'],
                "score": int(row['score']),
                "filename": row['filename'],
                "enhanced_description": row.get('enhanced_description', ''),
                "image_path": str(self.images_path / row['filename'])
            }
            results.append(result)

        return results

class ColabSearchInterface:
    """Interactive search interface for Colab"""

    def __init__(self, search_engine: ColabGreentextSearch):
        self.search_engine = search_engine
        self.current_results = []
        self.setup_interface()

    def setup_interface(self):
        """Setup the interactive widgets"""
        # Search input
        self.search_input = widgets.Text(
            value='',
            placeholder='Enter your search query (e.g., "wholesome story", "anon at work")',
            description='Search:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='70%')
        )

        # Search type dropdown
        self.search_type = widgets.Dropdown(
            options=[('Smart Search (Combined)', 'combined'),
                    ('Text Only', 'text'),
                    ('Visual Only', 'image')],
            value='combined',
            description='Type:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='25%')
        )

        # Search button
        self.search_button = widgets.Button(
            description='🔍 Search',
            button_style='primary',
            layout=widgets.Layout(width='15%')
        )

        # Results per page
        self.results_per_page = widgets.IntSlider(
            value=12,
            min=6,
            max=24,
            step=6,
            description='Results:',
            style={'description_width': 'initial'}
        )

        # Random button
        self.random_button = widgets.Button(
            description='🎲 Random',
            button_style='info'
        )

        # Output area
        self.output_area = widgets.Output()

        # Bind events
        self.search_button.on_click(self.on_search_click)
        self.random_button.on_click(self.on_random_click)
        self.search_input.on_submit(self.on_search_submit)

        # Show stats on startup
        with self.output_area:
            self.show_stats()

    def on_search_click(self, button):
        """Handle search button click"""
        self.perform_search()

    def on_search_submit(self, text_widget):
        """Handle search input submission"""
        self.perform_search()

    def on_random_click(self, button):
        """Handle random button click"""
        self.show_random_samples()

    def perform_search(self):
        """Perform search and display results"""
        query = self.search_input.value.strip()
        if not query:
            return

        with self.output_area:
            clear_output(wait=True)
            print(f"🔍 Searching for: '{query}'...")

            # Perform search
            results = self.search_engine.search(
                query,
                top_k=self.results_per_page.value,
                search_type=self.search_type.value
            )

            if results:
                print(f"✅ Found {len(results)} results\n")
                self.display_results(results, query)
            else:
                print("❌ No results found. Try a different query.")

    def show_random_samples(self):
        """Show random samples"""
        with self.output_area:
            clear_output(wait=True)
            print("🎲 Loading random samples...")

            results = self.search_engine.get_random_samples(self.results_per_page.value)

            if results:
                print(f"✅ Showing {len(results)} random samples\n")
                self.display_results(results, "Random Samples")

    def show_stats(self):
        """Display dataset statistics"""
        stats = self.search_engine.get_stats()

        print("📊 DATASET STATISTICS")
        print("=" * 50)
        print(f"📸 Total Images: {stats['total_images']:,}")
        print(f"📈 Average Score: {stats['average_score']:.1f}")
        print(f"🏆 Max Score: {stats['max_score']:,}")
        print(f"📉 Min Score: {stats['min_score']:,}")
        print(f"🎯 Median Score: {stats['median_score']:.1f}")
        print(f"🧠 Embedding Dimensions: {stats['embedding_dimensions']}")
        print("\n" + "=" * 50)
        print("💡 Try searching for: 'wholesome', 'work stories', 'mom', 'autism', 'feels'")
        print("=" * 50 + "\n")

    def display_results(self, results: List[Dict], query: str):
        """Display search results in a grid"""
        # Calculate grid dimensions
        n_cols = 3
        n_rows = (len(results) + n_cols - 1) // n_cols

        # Create figure
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        if n_rows == 1:
            axes = axes.reshape(1, -1)

        # Remove all axes initially
        for i in range(n_rows):
            for j in range(n_cols):
                axes[i, j].axis('off')

        # Display results
        for idx, result in enumerate(results):
            row = idx // n_cols
            col = idx % n_cols
            ax = axes[row, col]

            # Load and display image
            try:
                if Path(result['image_path']).exists():
                    img = mpimg.imread(result['image_path'])
                    ax.imshow(img)
                else:
                    # Create placeholder
                    ax.text(0.5, 0.5, 'Image\nNot Found',
                           ha='center', va='center', fontsize=16,
                           bbox=dict(boxstyle='round', facecolor='lightgray'))
                    ax.set_xlim(0, 1)
                    ax.set_ylim(0, 1)

                # Add title and info
                title = result['title'][:50] + "..." if len(result['title']) > 50 else result['title']

                info_text = f"Score: {result['score']:,}"
                if 'similarity_score' in result:
                    info_text += f" | Match: {result['similarity_score']*100:.1f}%"

                ax.set_title(f"{title}\n{info_text}",
                           fontsize=10, pad=10, wrap=True)
                ax.axis('off')

            except Exception as e:
                ax.text(0.5, 0.5, f'Error loading\nimage: {str(e)[:30]}',
                       ha='center', va='center', fontsize=10)
                ax.set_xlim(0, 1)
                ax.set_ylim(0, 1)
                ax.axis('off')

        plt.tight_layout()
        plt.show()

        # Display text information
        print("\n📝 DETAILED RESULTS:")
        print("-" * 80)
        for i, result in enumerate(results[:6]):  # Show details for first 6
            similarity_info = ""
            if 'similarity_score' in result:
                similarity_info = f" (Match: {result['similarity_score']*100:.1f}%)"

            print(f"{i+1}. {result['title']}{similarity_info}")
            print(f"   👍 {result['score']:,} upvotes | ID: {result['post_id']}")

            # Show enhanced description preview
            desc = result.get('enhanced_description', '')
            if desc:
                desc_preview = desc.split('|')[0][:100] + "..." if len(desc.split('|')[0]) > 100 else desc.split('|')[0]
                print(f"   📄 {desc_preview}")
            print()

    def display(self):
        """Display the complete interface"""
        # Title
        title_html = """
        <div style="text-align: center; padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 10px; margin-bottom: 20px;">
            <h1 style="margin: 0; font-size: 2.5em;">🚀 GreentextSearch</h1>
            <p style="margin: 10px 0 0 0; font-size: 1.2em;">AI-Powered Semantic Search for Greentext Memes</p>
        </div>
        """
        display(HTML(title_html))

        # Search controls
        search_box = widgets.HBox([
            self.search_input,
            self.search_type,
            self.search_button
        ])

        controls_box = widgets.HBox([
            self.results_per_page,
            self.random_button
        ])

        display(search_box)
        display(controls_box)
        display(self.output_area)

# ============== USAGE FUNCTIONS ==============

def initialize_search_engine(processed_data_path: str, images_path: str):
    """Initialize the search engine"""
    print("🚀 INITIALIZING GREENTEXT SEARCH ENGINE")
    print("=" * 60)

    try:
        search_engine = ColabGreentextSearch(processed_data_path, images_path)
        return search_engine
    except Exception as e:
        print(f"❌ Failed to initialize search engine: {e}")
        return None

def create_search_interface(search_engine: ColabGreentextSearch):
    """Create and display the search interface"""
    if search_engine is None:
        print("❌ Search engine not initialized")
        return None

    interface = ColabSearchInterface(search_engine)
    interface.display()
    return interface

# ============== QUICK SETUP FUNCTION ==============

def setup_greentext_search(processed_data_path=None, images_path=None):
    """
    Quick setup function for Colab users

    Args:
        processed_data_path: Path to processed data directory
        images_path: Path to images directory
    """

    # Default paths (update these to match your setup)
    if processed_data_path is None:
        processed_data_path = "/content/drive/MyDrive/greentext_ml_dataset/metadata/processed"

    if images_path is None:
        images_path = "/content/drive/MyDrive/greentext_ml_dataset/images"

    print("🔧 SETTING UP GREENTEXT SEARCH")
    print("=" * 50)
    print(f"📁 Processed data path: {processed_data_path}")
    print(f"🖼️ Images path: {images_path}")
    print()

    # Initialize search engine
    search_engine = initialize_search_engine(processed_data_path, images_path)

    if search_engine:
        print("\n🎉 SUCCESS! Creating search interface...")
        interface = create_search_interface(search_engine)
        return search_engine, interface
    else:
        print("\n❌ Failed to setup search engine")
        return None, None

# ============== EXAMPLE USAGE ==============
"""
# To use this in your Colab notebook:

# 1. Run this cell to load the code
# 2. Then run:

search_engine, interface = setup_greentext_search()

# The interface will appear and you can start searching!
# Try queries like:
# - "wholesome story"
# - "anon at work"
# - "mom"
# - "autism"
# - "feels"
"""

# Auto-setup if run directly
if __name__ == "__main__":
    # This will run automatically when you execute the cell
    search_engine, interface = setup_greentext_search()

🔧 SETTING UP GREENTEXT SEARCH
📁 Processed data path: /content/drive/MyDrive/greentext_ml_dataset/metadata/processed
🖼️ Images path: /content/drive/MyDrive/greentext_ml_dataset/images

🚀 INITIALIZING GREENTEXT SEARCH ENGINE
🖥️ Using device: cuda
📦 Loading CLIP model...


100%|███████████████████████████████████████| 338M/338M [00:07<00:00, 46.2MiB/s]


✅ CLIP model loaded successfully
📊 Loaded 898 records
🧠 Loaded embeddings - Shape: (898, 512)
🔍 FAISS indices built successfully
🚀 Search engine ready!

🎉 SUCCESS! Creating search interface...


Output()